# Notebook for data analysis

We will use Pandas library to get insights for data collected during simulations.

Created using python 3.10.12

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data_dir = "factory-sim/simulations/"
file_name = "output.csv"
df = pd.read_csv(data_dir + file_name)

/home/erni/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


Production times of non-defective items

In [3]:
df

,counter,discarded,gen_time,arrival_time,discard_time,prod_time
0,0,1,0.000000,0.000000,0.506726,-1.0
1,0,1,0.564257,0.564257,1.395410,-1.0
2,3,1,0.506726,0.506726,2.202010,-1.0
3,3,1,1.191840,1.395410,3.300840,-1.0
4,3,1,1.233300,1.418570,3.598840,-1.0
...,...,...,...,...,...,...
14393,2,1,2292.550000,7240.360000,7242.780000,-1.0
14394,3,1,2292.590000,7240.950000,7243.850000,-1.0
14395,3,1,2292.800000,7241.810000,7244.150000,-1.0
14396,3,1,2292.830000,7241.990000,7244.380000,-1.0
